In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY']= os.getenv('TAVILY_API_KEY')

In [2]:
import json

file_path = 'data.json'  

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [9]:
def extract_spending_details_by_category(data):
    categories = ['본식', '스드메', '예복', '예물/예단', '신혼여행', '가전/가구', '기타']
    
    result = {category: [] for category in categories}
    
    for category_data in data['groupedSpendings']:
        category_name = category_data['budgetCategory']['budgetCategoryName']
        # totalAmount은 사용 x
        #ipdb.set_trace()
        
        if category_name in categories:
            for spending in category_data['spendings']:
                spending_details = {
                    f"{spending['title']}": spending['amount']
                }
                result[category_name].append(spending_details)
    
    return result


extracted_budget_data = extract_spending_details_by_category(data)

print(extracted_budget_data)

{'본식': [{'식대': 0}, {'홀 대관료': 1000000}], '스드메': [{'스드메': 1439000}], '예복': [{'예복': 0}], '예물/예단': [{'결혼반지': 2390000}], '신혼여행': [{'숙박비': 0}, {'항공권': 0}, {'투어 및 기타 경비': 0}], '가전/가구': [{'청소기': 759000}, {'침대': 8102000}, {'삼성': 8770000}, {'식탁': 722000}, {'쇼파': 3204820}], '기타': [{'줄눈': 450000}, {'탄성코트': 550000}, {'하자점검': 270000}, {'입주청소': 510000}, {'TV(무타공)': 330000}]}


In [7]:
# English 
PROMPT_TEMPLATE_EG = """
# Instruction 
You are an AI assistant helping with wedding budget management. Please respond to the user's questions kindly and accurately. Your goal is to provide helpful advice based on the user’s current wedding budget, suggest possible savings or cost-efficient alternatives, and, when needed, provide additional information by searching the internet for price comparisons, recommendations, or tips.

Current wedding budget information:
{budget_info}


# GUIDELINES:

1. Always consider the user’s wedding budget limitations and provide cost-efficient suggestions where possible.
2. When there are zero values or missing information in certain categories, offer advice on possible cost ranges or alternatives based on typical wedding expenditures.
3. For large expenses, help the user identify areas where they might save by comparing current prices online or recommending alternative options (e.g., for furniture or honeymoon).
4. If there are optional categories, ask the user if they would like to explore further options for those categories (e.g., entertainment, videography).
5. Only respond to queries that are directly related to wedding budget management or planning.
6. Keep your tone friendly, empathetic, and professional.

# IMPORTANT : Answer in KOREAN

# Begin!

Previous conversation history:
{chat_history}

New input: {input}

"""

### [To-do] 예산 관련 정보 데이터 수집 -> RAG 구현